In [1]:
from pandas import read_csv

In [2]:
# Criando um objeto Pandas
uberFile = read_csv("data/uber.csv")

In [3]:
type(uberFile)

pandas.core.frame.DataFrame

In [4]:
# Visualizando as primeiras linhas
uberFile.head(10)

,dispatching_base_number,date,active_vehicles,trips
0,B02512,1/1/2015,190,1132
1,B02765,1/1/2015,225,1765
2,B02764,1/1/2015,3427,29421
3,B02682,1/1/2015,945,7679
4,B02617,1/1/2015,1228,9537
5,B02598,1/1/2015,870,6903
6,B02598,1/2/2015,785,4768
7,B02617,1/2/2015,1137,7065
8,B02512,1/2/2015,175,875
9,B02682,1/2/2015,890,5506


In [5]:
# Tranformando o dataframe (Pandas) em um Dataframe (Spark)
uberDF = sqlContext.createDataFrame(uberFile)

In [6]:
type(uberDF)

pyspark.sql.dataframe.DataFrame

In [7]:
# Criando o RDD a partir do arquivo csv
uberRDD = sc.textFile("data/uber.csv")

In [8]:
type(uberRDD)

pyspark.rdd.RDD

In [9]:
# Total de registros
uberRDD.count()

355

In [10]:
# Primeiro registro
uberRDD.first()

'dispatching_base_number,date,active_vehicles,trips'

In [11]:
# 1- Quantos são e quais são as bases de carros do Uber (onde os carros ficam esperando passageiros)?

# Quantos são?
uberDF.select('dispatching_base_number').distinct().count()

6

In [12]:
# Quais são?
uberDF.select('dispatching_base_number').distinct().show()

+-----------------------+
|dispatching_base_number|
+-----------------------+
|                 B02512|
|                 B02598|
|                 B02682|
|                 B02765|
|                 B02617|
|                 B02764|
+-----------------------+



In [13]:
# 2- Qual o total de veículos que passaram pela base B02617?

uberDF.filter(uberDF['dispatching_base_number'] == 'B02617').count()

59

In [14]:
# 3- Qual é o total de dias em que o número de corridas foi superior a 16000?

uberDF.filter(uberDF['trips'] > 16000).count()

66

In [15]:
# 4- Quais são os dias em que o total de corridas foi maior que 16000?
uberDF.filter(uberDF['trips'] > 16000).show()

+-----------------------+---------+---------------+-----+
|dispatching_base_number|     date|active_vehicles|trips|
+-----------------------+---------+---------------+-----+
|                 B02764| 1/1/2015|           3427|29421|
|                 B02764| 1/2/2015|           3147|19974|
|                 B02764| 1/3/2015|           3215|29729|
|                 B02764| 1/4/2015|           2862|20441|
|                 B02764| 1/5/2015|           3387|20926|
|                 B02764| 1/6/2015|           3473|25301|
|                 B02764| 1/7/2015|           3603|29949|
|                 B02764| 1/8/2015|           3831|33802|
|                 B02764| 1/9/2015|           3820|33517|
|                 B02764|1/10/2015|           3558|38864|
|                 B02764|1/11/2015|           3186|27681|
|                 B02764|1/12/2015|           3499|26852|
|                 B02764|1/13/2015|           3658|29983|
|                 B02764|1/14/2015|           3736|29550|
|             

In [17]:
# 5- Qual o total de corridas por base?

uberDF.groupBy(uberDF['dispatching_base_number']).agg({'active_vehicles':'sum'}).show()

+-----------------------+--------------------+
|dispatching_base_number|sum(active_vehicles)|
+-----------------------+--------------------+
|                 B02512|               13125|
|                 B02598|               58653|
|                 B02682|               71431|
|                 B02765|               22575|
|                 B02617|               79758|
|                 B02764|              217290|
+-----------------------+--------------------+

